## 1. RS, MaxSum, Exact algo

### RS

In [136]:
R_rs = 0.01:0.01:0.99
D_rs = [0.4546474073128681,0.43565049932361133,0.42092765046912317,0.40839439222208573,0.3972457420215144,0.3870702443196218,0.3776242651788484,0.36874955806591975,0.3603365352601175,0.3523056385673838,0.34459697668198624,0.33716414626734653,0.3299703418625086,0.3229857971761951,0.31618604029437997,0.30955066735206993,0.30306245811343413,0.2967067238049428,0.2904708168067388,0.2843437556958753,0.2783159341345032,0.27237889177802405,0.26652513178257226,0.2607479738209666,0.2550414345013817,0.24940012917780852,0.2438191906350483,0.2382942012118009,0.2328211357178961,0.22739631308980668,0.22201635516998064,0.21667815133009227,0.211378827914124,0.20688352899753953,0.20278317659003975,0.19868817793711935,0.1945975994415784,0.19051056552023182,0.18642625421128822,0.18234389327612,0.17826275673045133,0.17418216174918932,0.17010146589683228,0.16602006464188257,0.16193738911916555,0.15785290410864755,0.1537661062033548,0.14967652214243293,0.1455837072883871,0.14148724423013948,0.137998366107847,0.13448740638286893,0.13095421069018287,0.12739865696194796,0.12382065470933767,0.12022014422138577,0.11659709568295457,0.1129515082152911,0.10928340884389087,0.10559285139949881,0.10231362606715677,0.0989986151678956,0.09564794614910743,0.0922617930523803,0.0888403753926773,0.08538395674018595,0.0820151560863665,0.0788488704133426,0.07563678673548502,0.07237911796887381,0.06907615625262697,0.06591319349557895,0.06283584277258297,0.05970319879491964,0.056515457092013055,0.053562785457060835,0.05054512262289507,0.04752198628196386,0.04463641809759822,0.04167585667427448,0.0388827836051418,0.03604608454052277,0.033304960430999175,0.03060809220840449,0.027952952887639493,0.025359146185103376,0.022832861932172044,0.020370784947590925,0.017979055383501308,0.015666129177374533,0.013449978144655372,0.0113201258591068,0.009302254807509736,0.007397829094439923,0.0056264302423451595,0.004010648891537405,0.002577264143732827,0.0013672330158682189,0.0004509986577261871];

In [137]:
Rs = 0.15:0.05:0.7
Λ = [0, 1]
Ks = [zeros(0) for _ in Rs]
for (i,R) in enumerate(Rs)
    α = 1-R
    k = floor(Int, 2/α)
    s = k+1-2/α
    K = [fill(0,k-1); s; 1-s]
    Ks[i] = K ./ sum(K)
end

### Exact algo

In [138]:
function f(n, Rs, navg_exact, Ks, Λ, D_exact, verbose=true)
    for i in eachindex(Rs)
        m = round(Int,n*(1-Rs[i])); nedges = 2n
        Threads.@threads for j in 1:navg_exact
            H = permutedims(ldpc_matrix(n, m, nedges, Λ, Ks[i]))
            s = rand((-1,1),n)
            ovl, _, _ = findsol(H, s, verbose=false)
            D_exact[i][j] = (1-ovl)/2
        end
        verbose && println("Rate $(Rs[i]) done")
        flush(stdout)
    end
    mean.(D_exact)
end    

f (generic function with 2 methods)

In [139]:
n = 1200
navg_exact = 500
D_exact = [zeros(navg_exact) for _ in Rs]
D_exact_avg = f(n, Rs, navg_exact, Ks, Λ, D_exact);

Rate 0.15 done
Rate 0.2 done
Rate 0.25 done
Rate 0.3 done
Rate 0.35 done
Rate 0.4 done
Rate 0.45 done
Rate 0.5 done
Rate 0.55 done
Rate 0.6 done
Rate 0.65 done
Rate 0.7 done


### MaxSum

In [547]:
function ff(n, Rs, navg, Ks, Λ, D_ms, D_exact; verbose=true, maxiter=10^2, kw...)
    efield = [(.0,.0) for _ in 1:n]
    s = zeros(Int, n)
    for i in eachindex(Rs)
        m = round(Int,n*(1-Rs[i])); nedges = 2n
        vars = rand(1:n, n*2÷3); factors=rand(1:m-1, m*2÷3)
        for j in 1:navg
            Ht = ldpc_matrix(n, m, nedges, Λ, Ks[i]; accept_multi_edges=false)[:,1:end-1]
            H = permutedims(Ht)
            s .= rand((-1.,1.),n)
            efield .= [(si,-si).+ 1e-5.*(randn(),randn()) for si in s]
            ms = BPFull(H, efield)
            ε, iters = iteration_ms!(ms, maxiter=maxiter;
                vars=vars, factors=factors, kw...)
            @show nunsat, ovl, dist = performance(ms, s)
            flush(stdout)
            if nunsat!=0
                B,indep = findbasis(H, Ht)
                x = argmax.(ms.belief) .== 2
                σ = fix_indep!(x, B, indep)   
                D_ms[i][j] = distortion(σ,s)
                verbose && @show D_ms[i][j], dist
            else
                D_ms[i][j] = dist
                verbose && println("R=", Rs[i], ". avg $j of $navg. Converged")
            end
            ovl, _, _ = findsol(H, s, verbose=false)
            D_exact[i][j] = (1-ovl)/2
        end
        println("Rate $(Rs[i]) done")
        flush(stdout)
    end
    mean.(D_ms), mean.(D_exact)
end

ff (generic function with 1 method)

In [544]:
include("../code/bp_full.jl")
include("../code/optimal_cycle.jl")
include("../code/rs.jl")
include("../code/bp.jl")

cb_decimation (generic function with 2 methods)

In [ ]:
n = 1200
navg = 30
D_ms = [zeros(navg) for _ in Rs]
D_exact = [zeros(navg) for _ in Rs]

D_ms_avg, D_exact_avg = ff(n, Rs, navg, Ks, Λ, D_ms, D_exact, maxiter=10^3, rein=1e-3, verbose=false);

(nunsat, ovl, dist) = performance(ms, s) = (2, 0.36, 0.32)
(nunsat, ovl, dist) = performance(ms, s) = (1, 0.3566666666666667, 0.32166666666666666)
(nunsat, ovl, dist) = performance(ms, s) = (1, 0.3783333333333333, 0.31083333333333335)
(nunsat, ovl, dist) = performance(ms, s) = (1, 0.365, 0.3175)
(nunsat, ovl, dist) = performance(ms, s) = (4, 0.385, 0.3075)
(nunsat, ovl, dist) = performance(ms, s) = (1, 0.375, 0.3125)
(nunsat, ovl, dist) = performance(ms, s) = (1, 0.385, 0.3075)
(nunsat, ovl, dist) = performance(ms, s) = (2, 0.365, 0.3175)
(nunsat, ovl, dist) = performance(ms, s) = (2, 0.3533333333333334, 0.3233333333333333)
(nunsat, ovl, dist) = performance(ms, s) = (1, 0.3616666666666667, 0.31916666666666665)
(nunsat, ovl, dist) = performance(ms, s) = (2, 0.35, 0.325)
(nunsat, ovl, dist) = performance(ms, s) = (4, 0.365, 0.3175)
(nunsat, ovl, dist) = performance(ms, s) = (1, 0.3716666666666667, 0.31416666666666665)
(nunsat, ovl, dist) = performance(ms, s) = (2, 0.3583333333333333, 0.3

(nunsat, ovl, dist) = performance(ms, s) = (2, 0.54, 0.23)
(nunsat, ovl, dist) = performance(ms, s) = (3, 0.5366666666666666, 0.23166666666666666)
(nunsat, ovl, dist) = performance(ms, s) = (1, 0.5266666666666666, 0.23666666666666666)
(nunsat, ovl, dist) = performance(ms, s) = (3, 0.5683333333333334, 0.21583333333333332)
(nunsat, ovl, dist) = performance(ms, s) = (5, 0.5266666666666666, 0.23666666666666666)
(nunsat, ovl, dist) = performance(ms, s) = (3, 0.5349999999999999, 0.2325)
(nunsat, ovl, dist) = performance(ms, s) = (1, 0.5516666666666667, 0.22416666666666665)
(nunsat, ovl, dist) = performance(ms, s) = (1, 0.54, 0.23)
(nunsat, ovl, dist) = performance(ms, s) = (4, 0.5466666666666666, 0.22666666666666666)
(nunsat, ovl, dist) = performance(ms, s) = (1, 0.5416666666666667, 0.22916666666666666)
(nunsat, ovl, dist) = performance(ms, s) = (2, 0.5616666666666666, 0.21916666666666668)
(nunsat, ovl, dist) = performance(ms, s) = (0, 0.5416666666666667, 0.22916666666666666)
(nunsat, ovl, d

(nunsat, ovl, dist) = performance(ms, s) = (0, 0.6933333333333334, 0.15333333333333332)
Rate 0.45 done
(nunsat, ovl, dist) = performance(ms, s) = (1, 0.71, 0.145)
(nunsat, ovl, dist) = performance(ms, s) = (1, 0.7216666666666667, 0.13916666666666666)
(nunsat, ovl, dist) = performance(ms, s) = (1, 0.72, 0.14)
(nunsat, ovl, dist) = performance(ms, s) = (2, 0.7183333333333333, 0.14083333333333334)
(nunsat, ovl, dist) = performance(ms, s) = (1, 0.7066666666666667, 0.14666666666666667)
(nunsat, ovl, dist) = performance(ms, s) = (2, 0.7, 0.15)
(nunsat, ovl, dist) = performance(ms, s) = (1, 0.7050000000000001, 0.1475)
(nunsat, ovl, dist) = performance(ms, s) = (3, 0.7266666666666667, 0.13666666666666666)
(nunsat, ovl, dist) = performance(ms, s) = (5, 0.7150000000000001, 0.1425)
(nunsat, ovl, dist) = performance(ms, s) = (3, 0.7050000000000001, 0.1475)
(nunsat, ovl, dist) = performance(ms, s) = (3, 0.7216666666666667, 0.13916666666666666)
(nunsat, ovl, dist) = performance(ms, s) = (1, 0.718333

### Plot

In [ ]:
pl = plot_rdb(; f30=false, f3=false)
Plots.default(ms=3, msw=0, fontfamily="serif-roman", grid=false)
plot!(pl, R_rs[1:end-3], D_rs[1:end-3], label="RS", line=:dash, c=:gray)
scatter!(pl, Rs, D_exact_avg, label="Exact", m=:circle, c=:red, ms=4.5)
scatter!(pl, Rs, D_ms_avg, label="MaxSum", c=:green, m=:diamond)
plot!(pl, size=(400,400))

In [181]:
savefig("rdb_degree2.pdf")

In [489]:
m = round(Int,n*(1-Rs[i])); nedges = 2n
vars = rand(1:n, n*2÷3); factors=rand(1:m, m*2÷3)
H = permutedims(ldpc_matrix(n, m, nedges, Λ, Ks[i]; accept_multi_edges=false))
s = rand((-1.,1.),n)
efield = [(si,-si).+ 1e-5.*(randn(),randn()) for si in s]
ms = BPFull(H, efield)
@show ε, iters = iteration_ms!(ms, maxiter=maxiter, damp=0.0, tol=1e-12, rein=1e-4;
    vars=vars, factors=factors)
@show nunsat, ovl, dist = performance(ms, s)
    if nunsat!=0
        B,indep = findbasis(H[1:end-1,:])
        x = argmax.(ms.belief) .== 2
        σ = fix_indep!(x, B, indep) 
        @assert parity(H,x)==0               
        @show distortion(σ,s)
end
#     else
#         D_ms[i][j] = dist
#         println("R=", Rs[i], ". avg $j of $navg. Converged")
#     end
#         ovl, _, _ = findsol(H, s, verbose=false)


(ε, iters) = iteration_ms!(ms, maxiter = maxiter, damp = 0.0, tol = 1.0e-12, rein = 0.0001; vars = vars, factors = factors) = (0.0, 10000)
(nunsat, ovl, dist) = performance(ms, s) = (0, 0.3683333333333333, 0.31583333333333335)


In [338]:
i = 1
maxiter = 10^4
# const pa = fill(NaN, maxiter)
# const err = fill(NaN, maxiter)
# function cb(it, ε, ms)
#     pa[it] = parity(ms); err[it] = ε; false
# end
efield = [(1.,1.) for _ in 1:n]
m = round(Int,n*(1-Rs[i])); nedges = 2n
vars = rand(1:n, n*2÷3); factors=rand(1:m, m*2÷3)
@show vars[1], factors[1]
H = permutedims(ldpc_matrix(n, m, nedges, Λ, Ks[i]; accept_multi_edges=false))
s = rand((-1.,1.),n)
ms = BPFull(H, efield)
ms.efield .= [(si,-si).+ 1e-5.*(randn(),randn()) for si in s]
ms.h .= [h.+ 1e-5.*(randn(),randn()) for h in ms.h]
ms.u .= [u.+ 1e-5.*(randn(),randn()) for u in ms.u]
ε, iters = iteration_ms!(ms, maxiter=maxiter, damp=0.0, tol=1e-12, rein=1e-4, 
    vars=vars, factors=factors)
@show nunsat, ovl, dist = performance(ms, s)

(vars[1], factors[1]) = (342, 410)
(nunsat, ovl, dist) = performance(ms, s) = (4, 0.3866666666666667, 0.30666666666666664)


(4, 0.3866666666666667, 0.30666666666666664)

In [ ]:
function g(n, Rs, navg, Ks, Λ, ntrials, D_ms, D_exact; verbose=true, maxiter=10^2, kw...)
    efield = [(1.,1.) for _ in 1:n]   # pre-allocate
    for i in eachindex(Rs)
        m = round(Int,n*(1-Rs[i])); nedges = 2n
        vars = rand(1:n, n*2÷3); factors=rand(1:m, m*2÷3)
        Threads.@threads for j in 1:navg
            H = permutedims(ldpc_matrix(n, m, nedges, Λ, Ks[i]; accept_multi_edges=false))
            s = rand((-1.,1.),n)
            ms = BPFull(H, efield)
            for t in 1:ntrials
                ms.efield .= [(si,-si).+ 1e-5.*(randn(),randn()) for si in s]
                ms.h .= [(.5,.5).+ 1e-5.*(randn(),randn()) for h in ms.h]
                ms.u .= [(.5,.5).+ 1e-5.*(randn(),randn()) for u in ms.u]
                ε, iters = iteration_ms!(ms, maxiter=maxiter, damp=0.0, tol=1e-12, rein=1e-3;
                    vars=vars, factors=factors, kw...)
                @show nunsat, ovl, dist = performance(ms, s)
                if nunsat==0
                    D_ms[i][j] = dist
                    println("R=", Rs[i], ". avg $j of $navg_ms. Converged after $t trials")
                    ovl, _, _ = findsol(H, s, verbose=false)
                    D_exact[i][j] = (1-ovl)/2
                    flush(stdout)
                    break
                elseif t==ntrials
                    error("R=", Rs[i], ". avg $j of $navg. UNCONVERGED after $t trials")
                end
            end
        end
        verbose && println("Rate $(Rs[i]) done")
        flush(stdout)
    end
    mean.(D_ms), mean.(D_exact)
end